In [25]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

kaggle=False
model_path="microsoft/deberta-base"

In [26]:
path='/kaggle/input/clickbait-dataset/clickbait_data.csv' if kaggle else 'train.csv'

df=pd.read_csv(path)
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [27]:
for i in tqdm(range(len(df))):
    combined=""
    if str(df.loc[i, 'keyword']) != 'nan':
        combined += 'KEYWORD: ' + str(df.loc[i, 'keyword']).replace('%20', ' ') + ' | '
    if str(df.loc[i, 'location']) != 'nan':
        combined += 'LOCATION: ' + str(df.loc[i, 'location']) + ' | '
    combined += 'TEXT: ' + df.loc[i, 'text']
    df.loc[i, 'combined'] = combined

# Drop all but combined and target
df = df.drop(columns=['id', 'keyword', 'location', 'text'])

#rename target to label
df = df.rename(columns={'target': 'label'})
df.sample(30)


  0%|          | 0/7613 [00:00<?, ?it/s]

,label,combined
2644,1,KEYWORD: destruction | TEXT: So you have a new...
2227,0,KEYWORD: deluge | TEXT: The f$&amp;@ing things...
5448,1,KEYWORD: police | LOCATION: UK | TEXT: DT @geo...
132,0,KEYWORD: aftershock | TEXT: Aftershock back to...
6845,0,"KEYWORD: trauma | LOCATION: Montgomery County,..."
5559,0,KEYWORD: rainstorm | TEXT: @Calum5SOS you look...
1765,1,KEYWORD: collision | TEXT: my favorite lady ca...
1817,1,KEYWORD: crashed | TEXT: @brianroemmele UX fai...
6810,0,"KEYWORD: tragedy | LOCATION: Los Angeles, CA |..."
4398,1,"KEYWORD: hijacking | LOCATION: Athens,Greece |..."


In [28]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [29]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from datasets import Dataset

train_data=Dataset.from_pandas(train_df)
valid_data=Dataset.from_pandas(valid_df)

train_ds=train_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)
valid_ds=valid_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [31]:
train_ds[99]

{'label': 0,
 'combined': 'KEYWORD: explode | TEXT: Learn How I Gained Access To The Secrets Of The Top Earners &amp; Used Them To Explode My Home Business Here: http://t.co/SGXP1U5OL1 Please #RT',
 '__index_level_0__': 3398,
 'input_ids': [1,
  33754,
  771,
  11200,
  35,
  25931,
  1721,
  36367,
  35,
  10900,
  1336,
  38,
  48564,
  8076,
  598,
  20,
  34702,
  1525,
  20,
  3107,
  7535,
  268,
  359,
  3914,
  131,
  30374,
  28595,
  598,
  16161,
  4636,
  1308,
  2193,
  2090,
  1398,
  35,
  2054,
  640,
  90,
  4,
  876,
  73,
  33020,
  29269,
  134,
  791,
  245,
  3384,
  134,
  3401,
  849,
  13963,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [32]:
from transformers import Trainer, TrainingArguments



def accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=3,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    compute_metrics=accuracy
)

In [33]:
trainer.train()

c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2286 [00:00<?, ?it/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6873, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.694, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.03}
{'loss': 0.6936, 'learning_rate': 3e-06, 'epoch': 0.04}
{'loss': 0.6895, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6921, 'learning_rate': 5e-06, 'epoch': 0.07}
{'loss': 0.6833, 'learning_rate': 6e-06, 'epoch': 0.08}
{'loss': 0.6646, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.09}
{'loss': 0.638, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.1}
{'loss': 0.5671, 'learning_rate': 9e-06, 'epoch': 0.12}
{'loss': 0.5279, 'learning_rate': 1e-05, 'epoch': 0.13}


  0%|          | 0/191 [00:00<?, ?it/s]

...

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.4370654225349426, 'eval_accuracy': 0.835193696651346, 'eval_runtime': 2.5035, 'eval_samples_per_second': 608.353, 'eval_steps_per_second': 76.294, 'epoch': 2.76}
{'loss': 0.3705, 'learning_rate': 4.927211646136618e-06, 'epoch': 2.77}
{'loss': 0.3648, 'learning_rate': 4.647256438969766e-06, 'epoch': 2.78}
{'loss': 0.2407, 'learning_rate': 4.3673012318029114e-06, 'epoch': 2.8}
{'loss': 0.3872, 'learning_rate': 4.087346024636059e-06, 'epoch': 2.81}
{'loss': 0.3959, 'learning_rate': 3.8073908174692052e-06, 'epoch': 2.82}
{'loss': 0.428, 'learning_rate': 3.5274356103023515e-06, 'epoch': 2.83}
{'loss': 0.3743, 'learning_rate': 3.2474804031354986e-06, 'epoch': 2.85}
{'loss': 0.3938, 'learning_rate': 2.9675251959686453e-06, 'epoch': 2.86}
{'loss': 0.3283, 'learning_rate': 2.6875699888017915e-06, 'epoch': 2.87}
{'loss': 0.4209, 'learning_rate': 2.4076147816349386e-06, 'epoch': 2.89}


  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.44819602370262146, 'eval_accuracy': 0.8286277084701248, 'eval_runtime': 2.4915, 'eval_samples_per_second': 611.286, 'eval_steps_per_second': 76.662, 'epoch': 2.89}
{'loss': 0.5159, 'learning_rate': 2.1276595744680853e-06, 'epoch': 2.9}
{'loss': 0.3155, 'learning_rate': 1.847704367301232e-06, 'epoch': 2.91}
{'loss': 0.5238, 'learning_rate': 1.5677491601343787e-06, 'epoch': 2.93}
{'loss': 0.5116, 'learning_rate': 1.2877939529675251e-06, 'epoch': 2.94}
{'loss': 0.347, 'learning_rate': 1.0078387458006718e-06, 'epoch': 2.95}
{'loss': 0.3027, 'learning_rate': 7.278835386338187e-07, 'epoch': 2.97}
{'loss': 0.2201, 'learning_rate': 4.479283314669653e-07, 'epoch': 2.98}
{'loss': 0.4723, 'learning_rate': 1.6797312430011197e-07, 'epoch': 2.99}
{'train_runtime': 183.9435, 'train_samples_per_second': 99.324, 'train_steps_per_second': 12.428, 'train_loss': 0.4819971496664633, 'epoch': 3.0}


TrainOutput(global_step=2286, training_loss=0.4819971496664633, metrics={'train_runtime': 183.9435, 'train_samples_per_second': 99.324, 'train_steps_per_second': 12.428, 'train_loss': 0.4819971496664633, 'epoch': 3.0})

In [39]:
df_test=pd.read_csv('test.csv')
df_test['combined'] = 'KEYWORD: ' + df_test['keyword'].astype(str) + ' | LOCATION: ' + df_test['location'].astype(str) + ' | TEXT: ' + df_test['text'].astype(str)
df_test = df_test.drop(columns=['id', 'keyword', 'location', 'text'])
df_test = df_test.rename(columns={'target': 'label'})


test_data=Dataset.from_pandas(df_test)

test_ds=test_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)

model.eval()
preds=[]

preds=trainer.predict(test_ds)
preds=preds.predictions.argmax(-1)
df_submission_test_data=pd.read_csv('test.csv')
df_submission_test_data["target"]=preds
df_submission_test_data[["id", "target"]].to_csv("submission.csv", index=False)


Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

  0%|          | 0/408 [00:00<?, ?it/s]

In [ ]:
#81%